<a href="https://colab.research.google.com/github/greysat/SentimentPulse/blob/main/Scraping_data_from_GooglePlayStore_(FreeCell_Solitaire%2B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google_play_scraper #scraping reviews
!pip install -q transformers #sentiment
!pip install -q plotly-express #data visualization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00


In [2]:
!pip install pyyaml

In [3]:
import pandas as pd
import numpy as np
from google_play_scraper import app, reviews_all
import plotly.express as px

In [13]:
freecell_project = reviews_all( 'com.brainium.freecell', sleep_milliseconds=0, lang='en', country='US')

In [14]:
freecell_project

[{'reviewId': '3f3637bb-5f5f-4e44-bd9a-396fc30e7f1b',
  'userName': 'Michael Cleary',
  'userImage': 'https://play-lh.googleusercontent.com/a/ACg8ocINAn1C8fNJkbQ4i7aVK5L72mUyopGVpkO3Ow46gxpi=mo',
  'content': "I like the game. For a while, I was getting occasional advertisement even though I hadn't paid for ad-free version, but this has now been rectified. It is to the developer's credit that criticism was accepted and problem fixed. It gives me confidence in the developer.",
  'score': 5,
  'thumbsUpCount': 1,
  'reviewCreatedVersion': '1.5.19.163',
  'at': datetime.datetime(2023, 8, 5, 0, 39, 11),
  'replyContent': 'Hello Michael! This was a technical issue that has since been resolved. Thank you for your understanding and for playing FreeCell! 👑',
  'repliedAt': datetime.datetime(2023, 7, 28, 0, 28, 4),
  'appVersion': '1.5.19.163'},
 {'reviewId': '48bfbc13-9762-4111-8704-5c5f8d795101',
  'userName': 'John Tyler',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVZm0Na

In [15]:
df = pd.json_normalize(freecell_project)

In [16]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,3f3637bb-5f5f-4e44-bd9a-396fc30e7f1b,Michael Cleary,https://play-lh.googleusercontent.com/a/ACg8oc...,"I like the game. For a while, I was getting oc...",5,1,1.5.19.163,2023-08-05 00:39:11,Hello Michael! This was a technical issue that...,2023-07-28 00:28:04,1.5.19.163
1,48bfbc13-9762-4111-8704-5c5f8d795101,John Tyler,https://play-lh.googleusercontent.com/a-/ALV-U...,Great game and experience. Love that there are...,5,0,1.5.19.163,2023-08-03 09:32:04,None,NaT,1.5.19.163
2,cf8c25f3-be10-4764-8f08-f88b82649dc3,Cathy Singleton,https://play-lh.googleusercontent.com/a-/ALV-U...,"I paid for this app, and now you are pushing a...",4,0,1.5.19.163,2023-08-02 14:39:35,Hello Cathy! Make sure you have the correct ap...,2023-08-08 23:37:15,1.5.19.163
3,4c7e3f9b-48f1-4da9-a0ae-79899815dd47,David Cryans,https://play-lh.googleusercontent.com/a-/ALV-U...,Edit: Thank you for fixing it. \nOriginal: Sta...,3,0,1.5.19.163,2023-07-30 18:02:51,Hey David! This was a technical issue that has...,2023-07-28 00:30:57,1.5.19.163
4,5f46a5f6-c309-4993-83a5-b5336eb39dd3,Kevin Fung,https://play-lh.googleusercontent.com/a-/ALV-U...,"FreeCell Solitaire+ is worth the money, best f...",5,0,1.5.19.163,2023-07-29 07:48:50,Hello Kevin! This was a technical issue that h...,2023-07-28 00:23:57,1.5.19.163


In [17]:
df['score'].mean()

4.697452229299363

In [18]:
df['reviewCreatedVersion'].value_counts()

1.5.4.74      329
1.0.10        155
1.3.6.47      134
1.3.4.29      104
1.5.10.120     92
1.2.3          82
1.3.3          59
1.5.19.163     45
1.0.2          43
1.0.5          40
1.0.9          27
1.5.14.149     26
1.5.3.70       15
1.5.8.117      15
1.2            15
1.3.1           9
1.0.1           9
1.1             8
1.0.7           7
1.5.1.61        2
1.5.15.152      2
1.3.2           1
1.5.18.158      1
1.0             1
Name: reviewCreatedVersion, dtype: int64

In [20]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [21]:
df['content'] = df['content'].astype('str')

In [22]:
df['result'] = df['content'].apply(lambda x: sentiment_analysis(x))

In [23]:
data = df.to_dict(orient='records')

In [25]:
data = df.to_dict(orient='records')

In [26]:
df['sentiment']=df['result'].apply(lambda x:(x[0]['label']))
df['score']=df['result'].apply(lambda x:(x[0]['score']))

In [27]:
df['sentiment'].value_counts()

POSITIVE    1132
NEGATIVE     124
Name: sentiment, dtype: int64

In [28]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,result,sentiment
0,3f3637bb-5f5f-4e44-bd9a-396fc30e7f1b,Michael Cleary,https://play-lh.googleusercontent.com/a/ACg8oc...,"I like the game. For a while, I was getting oc...",0.998897,1,1.5.19.163,2023-08-05 00:39:11,Hello Michael! This was a technical issue that...,2023-07-28 00:28:04,1.5.19.163,"[{'label': 'POSITIVE', 'score': 0.998896718025...",POSITIVE
1,48bfbc13-9762-4111-8704-5c5f8d795101,John Tyler,https://play-lh.googleusercontent.com/a-/ALV-U...,Great game and experience. Love that there are...,0.998911,0,1.5.19.163,2023-08-03 09:32:04,None,NaT,1.5.19.163,"[{'label': 'POSITIVE', 'score': 0.998910665512...",POSITIVE
2,cf8c25f3-be10-4764-8f08-f88b82649dc3,Cathy Singleton,https://play-lh.googleusercontent.com/a-/ALV-U...,"I paid for this app, and now you are pushing a...",0.991423,0,1.5.19.163,2023-08-02 14:39:35,Hello Cathy! Make sure you have the correct ap...,2023-08-08 23:37:15,1.5.19.163,"[{'label': 'NEGATIVE', 'score': 0.991423070430...",NEGATIVE
3,4c7e3f9b-48f1-4da9-a0ae-79899815dd47,David Cryans,https://play-lh.googleusercontent.com/a-/ALV-U...,Edit: Thank you for fixing it. \nOriginal: Sta...,0.989918,0,1.5.19.163,2023-07-30 18:02:51,Hey David! This was a technical issue that has...,2023-07-28 00:30:57,1.5.19.163,"[{'label': 'POSITIVE', 'score': 0.989918053150...",POSITIVE
4,5f46a5f6-c309-4993-83a5-b5336eb39dd3,Kevin Fung,https://play-lh.googleusercontent.com/a-/ALV-U...,"FreeCell Solitaire+ is worth the money, best f...",0.998847,0,1.5.19.163,2023-07-29 07:48:50,Hello Kevin! This was a technical issue that h...,2023-07-28 00:23:57,1.5.19.163,"[{'label': 'POSITIVE', 'score': 0.998846530914...",POSITIVE


In [29]:
fig = px.histogram(df, x='sentiment', color='sentiment', text_auto=True)
fig.show()

In [30]:
data

[{'reviewId': '3f3637bb-5f5f-4e44-bd9a-396fc30e7f1b',
  'userName': 'Michael Cleary',
  'userImage': 'https://play-lh.googleusercontent.com/a/ACg8ocINAn1C8fNJkbQ4i7aVK5L72mUyopGVpkO3Ow46gxpi=mo',
  'content': "I like the game. For a while, I was getting occasional advertisement even though I hadn't paid for ad-free version, but this has now been rectified. It is to the developer's credit that criticism was accepted and problem fixed. It gives me confidence in the developer.",
  'score': 5,
  'thumbsUpCount': 1,
  'reviewCreatedVersion': '1.5.19.163',
  'at': Timestamp('2023-08-05 00:39:11'),
  'replyContent': 'Hello Michael! This was a technical issue that has since been resolved. Thank you for your understanding and for playing FreeCell! 👑',
  'repliedAt': Timestamp('2023-07-28 00:28:04'),
  'appVersion': '1.5.19.163',
  'result': [{'label': 'POSITIVE', 'score': 0.9988967180252075}]},
 {'reviewId': '48bfbc13-9762-4111-8704-5c5f8d795101',
  'userName': 'John Tyler',
  'userImage': 'htt